In [ ]:
!pip3 install transformers datasets

In [ ]:
import pandas as pd

df = pd.read_csv("QnA_Latest.csv")
df.index.name="id"
df

,question,Answer Span,context,Document,Keywords,Start index,Annotator
id,,,,,,,
0,Does an innocent recipient of UPSI have any de...,insider may prove his innocence by demonstrati...,The report (Para 55) suggests “where a person ...,1585217059979,"innocent recipient, UPSI, defence",1054.0,NaN
1,Will a promoter group entity require a pre-cle...,"only by ""Designated persons"" if the value of t...","With respect to the query at 5(i), attention m...",0c9f4131313e54e49d7b9915993ab00e,"pre-clearance, compliance officer, designated ...",678.0,NaN
2,Is inter-se off-market transfer of shares betw...,promoters have the option to convert warrants ...,"In the instant case, the said promoters have t...",0e8e621eb1aee80e102b4186ac874ed5,"inter-se off market transfer, block deal, cont...",0.0,NaN
3,Can an AIF invest its unutilized funds in liqu...,may invest investment income or investment pro...,The provisions under Regulation 15(1)(f) is pr...,1e4e946eca55a80467aad2e1546dd639,"unutilized funds, Alternate Investment Funds, ...",257.0,NaN
4,What are the penal consequences of not furnish...,attract the penalty prescribed under section 1...,The Honorable Securities Appellate Tribunal (h...,1289453383303,"Noticee, Summons, failure to Comply with summo...",1396.0,NaN
...,...,...,...,...,...,...,...
79,Who is a merchant banker?,any person who is engaged in the business of i...,“merchant banker” means any person who is eng...,Merchant Bankers,NaN,NaN,Vrinda
80,What is the capital adequacy requirement for a...,not less than five crore reupees,The capital adequacy requirement referred to i...,Merchant Bankers,NaN,NaN,Vrinda
81,"What is the definition of an ""issue""?",an offer of sale or purchase of securities by ...,“issue” means an offer of sale or purchase of...,Merchant Bankers,NaN,NaN,Vrinda


In [ ]:
def start_index(span,context):
  return context.find(span)
      
df["Start index"] = df.apply(lambda x: start_index(x["Answer Span"],x["context"]),axis=1)
df[["Answer Span","context","Start index"]]

,Answer Span,context,Start index
id,,,
0,insider may prove his innocence by demonstrati...,The report (Para 55) suggests “where a person ...,844
1,"only by ""Designated persons"" if the value of t...","With respect to the query at 5(i), attention m...",741
2,promoters have the option to convert warrants ...,"In the instant case, the said promoters have t...",30
3,may invest investment income or investment pro...,The provisions under Regulation 15(1)(f) is pr...,278
4,attract the penalty prescribed under section 1...,The Honorable Securities Appellate Tribunal (h...,248
...,...,...,...
79,any person who is engaged in the business of i...,“merchant banker” means any person who is eng...,25
80,not less than five crore reupees,The capital adequacy requirement referred to i...,-1
81,an offer of sale or purchase of securities by ...,“issue” means an offer of sale or purchase of...,15


In [ ]:
df[df["Start index"]==-1]

,question,Answer Span,context,Document,Keywords,Start index,Annotator
id,,,,,,,
80,What is the capital adequacy requirement for a...,not less than five crore reupees,The capital adequacy requirement referred to i...,Merchant Bankers,NaN,-1,Vrinda


In [ ]:
df = df[df["Start index"]!=-1]
df.reset_index()

df

,question,Answer Span,context,Document,Keywords,Start index,Annotator
id,,,,,,,
0,Does an innocent recipient of UPSI have any de...,insider may prove his innocence by demonstrati...,The report (Para 55) suggests “where a person ...,1585217059979,"innocent recipient, UPSI, defence",844,NaN
1,Will a promoter group entity require a pre-cle...,"only by ""Designated persons"" if the value of t...","With respect to the query at 5(i), attention m...",0c9f4131313e54e49d7b9915993ab00e,"pre-clearance, compliance officer, designated ...",741,NaN
2,Is inter-se off-market transfer of shares betw...,promoters have the option to convert warrants ...,"In the instant case, the said promoters have t...",0e8e621eb1aee80e102b4186ac874ed5,"inter-se off market transfer, block deal, cont...",30,NaN
3,Can an AIF invest its unutilized funds in liqu...,may invest investment income or investment pro...,The provisions under Regulation 15(1)(f) is pr...,1e4e946eca55a80467aad2e1546dd639,"unutilized funds, Alternate Investment Funds, ...",278,NaN
4,What are the penal consequences of not furnish...,attract the penalty prescribed under section 1...,The Honorable Securities Appellate Tribunal (h...,1289453383303,"Noticee, Summons, failure to Comply with summo...",248,NaN
...,...,...,...,...,...,...,...
78,Who are the witness when a search is being mad...,two or more independent and respectable inhabi...,"Before making a search under this Chapter, the...",Procedure for Search and Seizure,NaN,82,Vrinda
79,Who is a merchant banker?,any person who is engaged in the business of i...,“merchant banker” means any person who is eng...,Merchant Bankers,NaN,25,Vrinda
81,"What is the definition of an ""issue""?",an offer of sale or purchase of securities by ...,“issue” means an offer of sale or purchase of...,Merchant Bankers,NaN,15,Vrinda


In [ ]:
def preprocess(t,s):
  return {"answer_start":[s],"text":[t]}

df["answers"] = df.apply(lambda x: preprocess(x["Answer Span"],x["Start index"]),axis=1)
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,question,Answer Span,context,Document,Keywords,Start index,Annotator,answers
id,,,,,,,,
0,Does an innocent recipient of UPSI have any de...,insider may prove his innocence by demonstrati...,The report (Para 55) suggests “where a person ...,1585217059979,"innocent recipient, UPSI, defence",844,NaN,"{'answer_start': [844], 'text': ['insider may ..."
1,Will a promoter group entity require a pre-cle...,"only by ""Designated persons"" if the value of t...","With respect to the query at 5(i), attention m...",0c9f4131313e54e49d7b9915993ab00e,"pre-clearance, compliance officer, designated ...",741,NaN,"{'answer_start': [741], 'text': ['only by ""Des..."
2,Is inter-se off-market transfer of shares betw...,promoters have the option to convert warrants ...,"In the instant case, the said promoters have t...",0e8e621eb1aee80e102b4186ac874ed5,"inter-se off market transfer, block deal, cont...",30,NaN,"{'answer_start': [30], 'text': ['promoters hav..."
3,Can an AIF invest its unutilized funds in liqu...,may invest investment income or investment pro...,The provisions under Regulation 15(1)(f) is pr...,1e4e946eca55a80467aad2e1546dd639,"unutilized funds, Alternate Investment Funds, ...",278,NaN,"{'answer_start': [278], 'text': ['may invest i..."
4,What are the penal consequences of not furnish...,attract the penalty prescribed under section 1...,The Honorable Securities Appellate Tribunal (h...,1289453383303,"Noticee, Summons, failure to Comply with summo...",248,NaN,"{'answer_start': [248], 'text': ['attract the ..."
...,...,...,...,...,...,...,...,...
78,Who are the witness when a search is being mad...,two or more independent and respectable inhabi...,"Before making a search under this Chapter, the...",Procedure for Search and Seizure,NaN,82,Vrinda,"{'answer_start': [82], 'text': ['two or more i..."
79,Who is a merchant banker?,any person who is engaged in the business of i...,“merchant banker” means any person who is eng...,Merchant Bankers,NaN,25,Vrinda,"{'answer_start': [25], 'text': ['any person wh..."
81,"What is the definition of an ""issue""?",an offer of sale or purchase of securities by ...,“issue” means an offer of sale or purchase of...,Merchant Bankers,NaN,15,Vrinda,"{'answer_start': [15], 'text': ['an offer of s..."


In [ ]:
df=df.drop(["Start index"],axis=1)
df

,question,Answer Span,context,Document,Keywords,Annotator,answers
id,,,,,,,
0,Does an innocent recipient of UPSI have any de...,insider may prove his innocence by demonstrati...,The report (Para 55) suggests “where a person ...,1585217059979,"innocent recipient, UPSI, defence",NaN,"{'answer_start': [844], 'text': ['insider may ..."
1,Will a promoter group entity require a pre-cle...,"only by ""Designated persons"" if the value of t...","With respect to the query at 5(i), attention m...",0c9f4131313e54e49d7b9915993ab00e,"pre-clearance, compliance officer, designated ...",NaN,"{'answer_start': [741], 'text': ['only by ""Des..."
2,Is inter-se off-market transfer of shares betw...,promoters have the option to convert warrants ...,"In the instant case, the said promoters have t...",0e8e621eb1aee80e102b4186ac874ed5,"inter-se off market transfer, block deal, cont...",NaN,"{'answer_start': [30], 'text': ['promoters hav..."
3,Can an AIF invest its unutilized funds in liqu...,may invest investment income or investment pro...,The provisions under Regulation 15(1)(f) is pr...,1e4e946eca55a80467aad2e1546dd639,"unutilized funds, Alternate Investment Funds, ...",NaN,"{'answer_start': [278], 'text': ['may invest i..."
4,What are the penal consequences of not furnish...,attract the penalty prescribed under section 1...,The Honorable Securities Appellate Tribunal (h...,1289453383303,"Noticee, Summons, failure to Comply with summo...",NaN,"{'answer_start': [248], 'text': ['attract the ..."
...,...,...,...,...,...,...,...
78,Who are the witness when a search is being mad...,two or more independent and respectable inhabi...,"Before making a search under this Chapter, the...",Procedure for Search and Seizure,NaN,Vrinda,"{'answer_start': [82], 'text': ['two or more i..."
79,Who is a merchant banker?,any person who is engaged in the business of i...,“merchant banker” means any person who is eng...,Merchant Bankers,NaN,Vrinda,"{'answer_start': [25], 'text': ['any person wh..."
81,"What is the definition of an ""issue""?",an offer of sale or purchase of securities by ...,“issue” means an offer of sale or purchase of...,Merchant Bankers,NaN,Vrinda,"{'answer_start': [15], 'text': ['an offer of s..."


In [ ]:
df.to_csv("tempQnA.csv")


## Normal Predictions

In [ ]:
import ast
import csv
# create a dictionary
data = []
csvFilePath = 'tempQnA.csv'

# Open a csv reader called DictReader
with open(csvFilePath, encoding='utf-8') as csvf:
    csvReader = csv.DictReader(csvf)

    # jsonf= open(jsonFilePath, 'w', encoding='utf-8')
    for id,rows in enumerate(csvReader):
        temp=dict()
        temp["id"]=id
        temp["data"]=rows
        temp["data"]['answers']= ast.literal_eval(rows["answers"])
        data.append(temp)

In [ ]:
from datasets import load_metric

squad_v2 = False

metric = load_metric("squad_v2" if squad_v2 else "squad")

In [ ]:
from transformers import pipeline

model_name = "deepset/roberta-base-squad2"
model = pipeline(model=model_name, tokenizer=model_name, task="question-answering")


In [ ]:
formatted_predictions=list()
references=list()
for d in data:
  r=d["data"]
  result = model(question = r["question"],context=r["context"])
  references.append({"id": r["id"], "answers": r["answers"]})
  formatted_predictions.append({"id": r["id"], "prediction_text": result['answer']})



/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 14.457831325301205, 'f1': 49.043942827766564}

In [ ]:
print(formatted_predictions,file=open("Predictions/prediction_roberta.txt","w"))
print(references,file=open("Predictions/references.txt","w"))

In [ ]:
model_name = "mrm8488/longformer-base-4096-finetuned-squadv2"

model = pipeline(model=model_name, tokenizer=model_name, task="question-answering")


In [ ]:
formatted_predictions=list()
references=list()
for d in data:
  r=d["data"]
  result = model(question = r["question"],context=r["context"])
  references.append({"id": r["id"], "answers": r["answers"]})
  formatted_predictions.append({"id": r["id"], "prediction_text": result['answer']})



/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 13.25301204819277, 'f1': 43.975429245730886}

In [ ]:
print(formatted_predictions,file=open("Predictions/prediction_longformer.txt","w"))

In [ ]:
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"

model = pipeline(model=model_name, tokenizer=model_name, task="question-answering")


In [ ]:
formatted_predictions=list()
references=list()
for d in data:
  r=d["data"]
  result = model(question = r["question"],context=r["context"])
  references.append({"id": r["id"], "answers": r["answers"]})
  formatted_predictions.append({"id": r["id"], "prediction_text": result['answer']})



/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 15.662650602409638, 'f1': 53.10469389928735}

In [ ]:
print(formatted_predictions,file=open("Predictions/prediction_bert.txt","w"))

In [ ]:
model_name = "mrm8488/spanbert-finetuned-squadv1"

model = pipeline(model=model_name, tokenizer=model_name, task="question-answering")

Downloading:   0%|          | 0.00/493 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
formatted_predictions=list()
references=list()
for d in data:
  r=d["data"]
  result = model(question = r["question"],context=r["context"])
  references.append({"id": r["id"], "answers": r["answers"]})
  formatted_predictions.append({"id": r["id"], "prediction_text": result['answer']})



/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 10.843373493975903, 'f1': 44.32594934057122}

In [ ]:
print(formatted_predictions,file=open("Predictions/prediction_spanbert.txt","w"))

In [ ]:
model_name = "deepset/bert-base-cased-squad2"

model = pipeline(model=model_name, tokenizer=model_name, task="question-answering")


Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/152 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
formatted_predictions=list()
references=list()
for d in data:
  r=d["data"]
  result = model(question = r["question"],context=r["context"])
  references.append({"id": r["id"], "answers": r["answers"]})
  formatted_predictions.append({"id": r["id"], "prediction_text": result['answer']})


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 9.63855421686747, 'f1': 39.028056383518184}

In [ ]:
print(formatted_predictions,file=open("Predictions/prediction_deepsetBertbase.txt","w"))

## Outputs collations

In [ ]:
import re
collact = dict()
start="Predictions/"
for fn in ["prediction_bert.txt","prediction_longformer.txt","prediction_spanbert.txt","prediction_deepsetBertbase.txt","prediction_roberta.txt","references.txt"]:
    temp=start+fn
    with open(temp) as f:
        file = f.read().strip('][')
    try:
        name=fn.split("prediction_")[1]
        temp= re.compile("(?<=prediction_text':).*?'}")
        results = re.findall(temp,file)
        results =[r[2:-2] for r in results]
    except:
        name=fn
        temp= re.compile("(?<=text': ).*?]")
        results = re.findall(temp,file)
        results =[r[2:-2] for r in results]
        
    name=name.split(".txt")[0]
    collact[name]=results

In [ ]:
mo = pd.DataFrame(collact)

In [ ]:
mo["question"] = df["question"]
temp = mo.columns.tolist()
temp.remove("question")
temp.remove("references")
mo = mo[["question","references"]+temp]
mo

,question,references,bert,longformer,spanbert,deepsetBertbase,roberta
0,Does an innocent recipient of UPSI have any de...,insider may prove his innocence by demonstrati...,It was proposed not to bring such a defense in...,An insider may prove his innocence,An insider may prove his innocence,An insider may prove his innocence,An insider may prove his innocence
1,Will a promoter group entity require a pre-cle...,"only by ""Designated persons"" if the value of t...",will be required to obtain pre-clearance for t...,"if designated as a ""designated person"" by the ...",a promoter,will be required to obtain pre-clearance for t...,pre-clearance is required to be obtained only ...
2,Is inter-se off-market transfer of shares betw...,promoters have the option to convert warrants ...,will also attract the contra-trade restriction...,may attract the contra-trade restrictions,if the promoters or members of the promoter group,will also attract the contra-trade restriction...,contra-trade restrictions
3,Can an AIF invest its unutilized funds in liqu...,may invest investment income or investment pro...,may invest investment income or investment pro...,in liquid mutual funds,liquid mutual funds,liquid,may invest investment income
4,What are the penal consequences of not furnish...,attract the penalty prescribed under section 1...,penalty prescribed under section 15A of the SE...,the Investigating Authority of SEBI,the penalty,the penalty,penalty prescribed under section 15A of the SE...
...,...,...,...,...,...,...,...
78,Who are the witness when a search is being mad...,two or more independent and respectable inhabi...,two or more independent and respectable inhabi...,no such inhabitant of the said locality,two or more independent and respectable inhabi...,no such inhabitant of the said locality,two or more independent and respectable inhabi...
79,Who is a merchant banker?,any person who is engaged in the business of i...,any person who is engaged in the business of i...,any person who is engaged in the business of i...,any person who is engaged in the business of i...,any person who is engaged in the business of i...,any person who is engaged in the business of i...
80,NaN,an offer of sale or purchase of securities by ...,an offer of sale or purchase of securities,an offer of sale or purchase of securities,an offer of sale or purchase of securities by ...,an offer of sale or purchase of securities,an offer of sale or purchase of securities by ...
81,"What is the definition of an ""issue""?",person heading and discharging the finance fun...,the person heading and discharging the finance...,the person heading and discharging the finance...,the recognised stock exchange(s),whole time finance director,the person heading and discharging the finance...


In [ ]:
mo.to_csv("Predictions/ModelOutputs.csv",index=False)

In [ ]:
!zip -r results.zip Predictions/